In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, unix_timestamp, hour, to_date

In [ ]:

HIVE_URI = 'thrift://hive-metastore:9083'
MINIO_ACCESS_KEY = 'FrmF5fXO0bxpBepjVUSX'
MINIO_SECRET_KEY = '5RgPTIToiUPa16HAWnRv3KcsE7y21Oo3RPA3QXTb'

spark = SparkSession.builder \
    .appName("IcebergAsDefaultCatalog") \
    .config('spark.jars.packages', 
            'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,'
            'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,'
            'software.amazon.awssdk:bundle:2.17.178,'
            'software.amazon.awssdk:url-connection-client:2.17.178') \
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.spark_catalog.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://warehouse/") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.hive.metastore.sasl.enabled", "false") \
    .getOrCreate()



## Start Spark Session
print("Spark Running")


In [25]:
def load_bronze(file_name, table_name):
    """
    Load a CSV file into an Iceberg table in the bronze catalog.
    :param file_name: Name of the CSV file to load.
    :param table_name: Name of the Iceberg table to insert into.
    """

    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(f"s3a://test/bronze.{file_name}")

    df.write.format("iceberg") \
        .mode("overwrite") \
        .insertInto(f"spark_catalog.bronze.{table_name}")

In [26]:
# List of files and corresponding table names
files_to_load = [
    ("crm_cust_info.csv", "crm_cust_info"),
    ("crm_prd_info.csv", "crm_prd_info"),
    ("crm_sales_details.csv", "crm_sales_details"),
    ("erp_cust_az12.csv", "erp_cust_az12"),
    ("erp_loc_a101.csv", "erp_loc_a101"),
    ("erp_px_cat_g1v2.csv", "erp_px_cat_g1v2")
]

# Load each file into the corresponding Iceberg table
for file_name, table_name in files_to_load:
    load_bronze(file_name, table_name)

# Stop the Spark session
spark.stop()
print("Data loaded into Iceberg tables successfully.")
# Note: The above code assumes that the Iceberg tables and the S3 bucket structure already exist.

Data loaded into Iceberg tables successfully.
